<a href="https://colab.research.google.com/github/Nattakonjpg/AIPrototype2023/blob/main/Predict_MoViNets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Connect Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/Game analysis/Video/Vdo_model


/content/drive/.shortcut-targets-by-id/1jusOl_HN_QQ9kYMlGXffiLoyy9tUOXuM/Game analysis/Video/Vdo_model


##Install Package and Library

In [3]:
!pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 41.5 MB/s eta 0:00:00


In [4]:
pip install git+https://github.com/Atze00/MoViNet-pytorch.git

  Cloning https://github.com/Atze00/MoViNet-pytorch.git to /tmp/pip-req-build-gf4pl_ff
  Running command git clone --filter=blob:none --quiet https://github.com/Atze00/MoViNet-pytorch.git /tmp/pip-req-build-gf4pl_ff
  Resolved https://github.com/Atze00/MoViNet-pytorch.git to commit c2d1edf48fc6c5259707f9d833f22171b4f63493
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for MoViNet-pytorch: filename=MoViNet_pytorch-0.2.2-py3-none-any.whl size=11959 sha256=38ba4c1dea9c3859917be1480e12a7c6721987c15b113c6b8171937375a8d960
  Stored in directory: /tmp/pip-ephem-wheel-cache-qifx2_0n/wheels/a4/76/cf/b8a087460d770e5a44eff9ed975e225d43f007c8c14f8e26d2
  Created wheel for

In [7]:
import time
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
import torch
import transforms as T
from movinets import MoViNet
from movinets.config import _C

#Predict

##Install ffmpeg


In [ ]:
from IPython.display import clear_output
!sudo curl -L https://github.com/BtbN/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz -o /usr/local/bin/ffmpeg.tar.xz
clear_output()
%cd /usr/local/bin/
clear_output()
!7z e /usr/local/bin/ffmpeg.tar.xz
clear_output()
!7z e /usr/local/bin/ffmpeg.tar
clear_output()
!sudo chmod a+rx /usr/local/bin/ffmpeg
clear_output()
%cd /content/
!sudo curl -L https://mkvtoolnix.download/appimage/MKVToolNix_GUI-70.0.0-x86_64.AppImage -o /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo chmod u+rx /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo ln -s /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage /usr/local/bin/mkvmerge
!sudo chmod a+rx /usr/local/bin/mkvmerge
clear_output()
!ffmpeg -version

### **FFMPEG**
- Path Folder video ต้นฉบับ
- Path output video
- ตัดวิดีโอต้นฉบับให้เป็นวิดีโอละ 2 วินาที
- ตั้งชื่อวิดีโอ "video_test_{i+1}.avi"

In [ ]:
import os
import subprocess

# กำหนด path ของวิดีโอที่ต้องการตัด
inputFile = "/content/drive/MyDrive/Game analysis/Video/Vdo_model/video แบ่งรอบไม่ตัด/video_data/test_video_fullvideo/[8-30-23][03.05-33.00] nosyy - yoru - rank immortal 1 - รอบ 1.avi"

# เรียกใช้ ffmpeg เพื่อรับข้อมูลเกี่ยวกับวิดีโอและคำนวณความยาวของมัน
command = ['ffmpeg', '-i', inputFile, '-f', 'null', '-']
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
output = result.stdout.decode('utf-8')

# หาค่า duration จาก output
duration_index = output.find('Duration: ') + len('Duration: ')
duration = output[duration_index:duration_index+11]
duration = duration.split(':')
duration = int(duration[0])*3600 + int(duration[1])*60 + int(duration[2].split('.')[0])

# กำหนดความยาวของคลิปวิดีโอที่ต้องการตัด
clipDuration = 2  # คลิปวิดีโอละ 2 วินาที

# กำหนด path ที่ต้องการบันทึกไฟล์ผลลัพธ์
outputFolder = "/content/drive/MyDrive/Game analysis/Video/Vdo_model/video แบ่งรอบไม่ตัด/video_data/test_video_cutvideo/Immortal_1"

# คำนวณจำนวนคลิปวิดีโอที่ต้องการตัด
numClips = duration // clipDuration

# สร้างคำสั่ง FFMPEG สำหรับการตัดแต่ละคลิปวิดีโอ
for i in range(numClips):
    # คำนวณเวลาเริ่มต้นและสิ้นสุดของคลิปวิดีโอ
    startTime = i * clipDuration
    endTime = startTime + clipDuration

    # สร้าง command line สำหรับ FFMPEG
    commandLine = ['-hide_banner', '-i', inputFile, '-map', '0', '-c', 'copy', '-ss', "{:02d}:{:02d}:{:02d}".format(startTime // 3600, (startTime // 60) % 60, startTime % 60), '-to', "{:02d}:{:02d}:{:02d}".format(endTime // 3600, (endTime // 60) % 60, endTime % 60)]

    # กำหนด path ของไฟล์ผลลัพธ์
    outputFile = os.path.join(outputFolder, f"video_test_{i+1}.avi")
    commandLine += [outputFile]

    # ใช้ FFMPEG เพื่อตัดคลิปวิดีโอ
    subprocess.run(['ffmpeg'] + commandLine)


- สร้างไฟล์ text ที่ชื่อวิดีโอตรงกันและมี ID 2 หลัง.avi

In [ ]:
import os

# กำหนด path ของโฟลเดอร์ที่มีวิดีโอ
folder_path = "/content/drive/MyDrive/Game analysis/Video/Vdo_model/video แบ่งรอบไม่ตัด/video_data/test_video_cutvideo/Immortal_1/"

# สร้างไฟล์ text
output_file_path = "/content/drive/MyDrive/Game analysis/Video/Vdo_model/video แบ่งรอบไม่ตัด/test_train_splits/Immortal_1_test_split1.txt"
with open(output_file_path, "w") as f:
    # วนลูปผ่านไฟล์ในโฟลเดอร์
    for i, filename in enumerate(os.listdir(folder_path), start=1):
        # ตรวจสอบว่าไฟล์เป็นไฟล์วิดีโอ (.avi) หรือไม่
        if filename.endswith(".avi"):
            # สร้างชื่อไฟล์ใหม่
            new_filename = f"video_test_{i}.avi 2\n"
            # เขียนชื่อไฟล์ใหม่ลงในไฟล์ text
            f.write(new_filename)


## **Load Model**

In [8]:
%cd /content/drive/MyDrive/Game analysis/Video/Vdo_model/video แบ่งรอบไม่ตัด/video_data
%ls

/content/drive/.shortcut-targets-by-id/1jusOl_HN_QQ9kYMlGXffiLoyy9tUOXuM/Game analysis/Video/Vdo_model/video แบ่งรอบไม่ตัด/video_data
test_video_cutvideo/  test_video_fullvideo/


In [ ]:
# Define the model architecture
loaded_model1 = MoViNet(_C.MODEL.MoViNetA5, causal=False, pretrained=True)
loaded_model1.classifier[3] = torch.nn.Conv3d(2048, 8, (1, 1, 1))

# Specify the path to the checkpoint file
best_checkpoint_path = '/content/drive/MyDrive/Game analysis/Video/Vdo_model/Model/Model Yoru Checkpoint/A5_yoru+habor_Checkpoint_epoch_round112 (1).pth'

# Load the checkpoint, specifying map_location to CPU
best_checkpoint = torch.load(best_checkpoint_path, map_location=torch.device('cpu'))

# Load the model's state dictionary
loaded_model1.load_state_dict(best_checkpoint['model_state_dict'])

# Set the model to evaluation mode
loaded_model1.eval()


##Prepare DataLoader

In [10]:
torch.manual_seed(97)
num_frames = 24 # 16
clip_steps = 12
Bs_Train = 1
Bs_Test = 1

transform_test_A5 = transforms.Compose([
                                 T.ToFloatTensorInZeroOne(),
                                 T.Resize((400, 400)),
                                 #T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 T.CenterCrop((320, 320))]) #Size for model


hmdb51_test_A5 = torchvision.datasets.HMDB51('/content/drive/MyDrive/Game analysis/Video/Vdo_model/video แบ่งรอบไม่ตัด/video_data/test_video_cutvideo/',
                                             '/content/drive/MyDrive/Game analysis/Video/Vdo_model/video แบ่งรอบไม่ตัด/test_train_splits//',
                                              frames_per_clip=24,frame_rate=24,step_between_clips = clip_steps, fold=1, train=False,transform=transform_test_A5, num_workers=2)

test_loader_A5  = DataLoader(hmdb51_test_A5 , batch_size=Bs_Test, shuffle=False)

100%|██████████| 4/4 [00:18<00:00,  4.73s/it]


In [11]:
print(hmdb51_test_A5)

Dataset HMDB51
    Number of datapoints: 109
    Root location: /content/drive/MyDrive/Game analysis/Video/Vdo_model/video แบ่งรอบไม่ตัด/video_data/test_video_cutvideo/


## **Predict**

In [13]:
import torch
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from prettytable import PrettyTable
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report

###Predict Code

In [ ]:
# Make predictions
with torch.no_grad():
    output1 = loaded_model1(test_loader_A5)

logits1= output1  # Replace this with your actual output tensor
probabilities = F.softmax(logits1, dim=1)

# Get the predicted class index and its probability
predicted_class, highest_probability = torch.max(probabilities, dim=1)

print("Highest Probability:", highest_probability.item())
print("Predicted Class:", predicted_class.item())


In [ ]:
device = next(loaded_model1.parameters()).device

true_labels, predicted_labels = [], []

with torch.no_grad():
    for data, _, target in test_loader_A5:
        # Move data and target to the same device as the model
        data, target = data.to(device), target.to(device)

        output = F.log_softmax(loaded_model1(data), dim=1)
        _, predicted = torch.max(output, dim=1)

        true_labels.extend(target.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

In [14]:
# ประมวลผลข้อมูล validation ด้วยโมเดลที่คุณเตรียมไว้
device = next(loaded_model1.parameters()).device
true_labels, predicted_labels = [], []

# วนลูปผ่านชุดข้อมูล validation
with torch.no_grad():
    for data, _, target in test_loader_A5:
        # ย้ายข้อมูลและเป้าหมายไปยังอุปกรณ์เดียวกับโมเดล
        data, target = data.to(device), target.to(device)

        # ทำนายคลาสของข้อมูล
        output = F.log_softmax(loaded_model1(data), dim=1)
        _, predicted = torch.max(output, dim=1)

        # เก็บค่าเป้าหมายและค่าทำนาย
        true_labels.extend(target.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# แสดงผลลัพธ์ที่ได้
print("True Labels:", true_labels)
print("Predicted Labels:", predicted_labels)

/usr/local/lib/python3.10/dist-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


True Labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Predicted Labels: [0, 0, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 0, 7, 0, 6, 6, 6, 0, 0, 2, 2, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# Initialize lists to store true and predicted labels
true_labels, predicted_labels = [], []

# Define the device
device = next(loaded_model1.parameters()).device

# Iterate through the data loader
with torch.no_grad():
    for data, _, target in test_loader_A5:
        # Move data and target to the same device as the model
        data, target = data.to(device), target.to(device)

        # Perform the prediction
        output = F.log_softmax(loaded_model1(data), dim=1)
        _, predicted = torch.max(output, dim=1)

        # Extend the true and predicted labels lists
        true_labels.extend(target.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Convert lists to sets to get unique class labels
true_classes = set(true_labels)
predicted_classes = set(predicted_labels)

# Determine the correctly identified classes
correct_classes = true_classes.intersection(predicted_classes)

print("Correctly identified classes:")
print(correct_classes)


### Save predict to csv

In [16]:
import pandas as pd
from google.colab import drive

# Mount Google Drive เพื่อให้สามารถเข้าถึงไดเรกทอรีบน Google Drive
drive.mount('/content/drive')

# สร้าง DataFrame จาก true_labels และ predicted_labels
df = pd.DataFrame({'true_label': true_labels, 'predicted_label': predicted_labels})

# บันทึก DataFrame เป็นไฟล์ CSV ในเส้นทางที่กำหนด
output_path = '/content/drive/MyDrive/Game analysis/Video/Vdo_model/เก็บ csv ผลโมเดล/true_predict_A5_yoru+habor_2_with 1 round(ครั้งที่2).csv'
df.to_csv(output_path, index=False)

# บันทึก DataFrame เป็นไฟล์ CSV
df.to_csv('true_predict_A5_yoru+habor_2_with 1 round(ครั้งที่2)..csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
df

,true_label,predicted_label
0,0,0
1,0,0
2,0,0
3,0,7
4,0,7
...,...,...
104,0,0
105,0,0
106,0,0
107,0,0


### Mapping Predict_labe กับ Class Model

In [19]:
class_names = ['Buy action', 'Class 0 action', 'Dead Action', 'Defuse Action', 'Orb Action', 'Plant Action', 'Shoot Action', 'Skill Action']

In [ ]:
# สร้าง list เพื่อเก็บชื่อคลาสที่สอดคล้องกับลำดับของ predicted_labels
predicted_class_names = [class_names[label] for label in df['predicted_label']]

# เพิ่มคอลัมน์ใหม่ใน DataFrame เพื่อเก็บชื่อคลาสที่พยายามทำนาย
df['predicted_class_names'] = predicted_class_names


In [24]:
df

,true_label,predicted_label,predicted_class_names
0,0,0,Buy action
1,0,0,Buy action
2,0,0,Buy action
3,0,7,Skill Action
4,0,7,Skill Action
...,...,...,...
104,0,0,Buy action
105,0,0,Buy action
106,0,0,Buy action
107,0,0,Buy action


In [30]:
# หาความถี่ของแต่ละคลาสในคอลัมน์ predicted_labels
class_frequency = df['predicted_label'].value_counts()

# แสดงผลลัพธ์
print(class_frequency)


0    76
7    17
2    11
6     4
1     1
Name: predicted_label, dtype: int64
